# Adobe research unit

In [1]:
import os
from getpass import getpass

from dotenv import load_dotenv



In [2]:
import pstuts_rag

In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

load_dotenv()

def set_api_key_if_not_present(key_name, prompt_message=""):
    if len(prompt_message) == 0:
        prompt_message=key_name
    if key_name not in os.environ or not os.environ[key_name]:
        os.environ[key_name] = getpass.getpass(prompt_message)

set_api_key_if_not_present("OPENAI_API_KEY")
set_api_key_if_not_present("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE - MBUDISIC - CERT - {unique_id}"
set_api_key_if_not_present("LANGCHAIN_API_KEY")


In [5]:
from dataclasses import dataclass
@dataclass
class ApplicationParameters:
    filename = "data/test.json"
    embedding_model = "text-embedding-3-small"
    tool_calling_model = "gpt-4.1-mini"
    n_context_docs = 2

params = ApplicationParameters()

In [6]:

from langchain_core.messages import  HumanMessage
from langchain_openai.chat_models import ChatOpenAI

from pstuts_rag.agents import PsTutsTeamState


In [7]:
test_query = PsTutsTeamState(messages=[HumanMessage(content="What are layers?")],
                             team_members=[], next="")

In [8]:
llm_tool_calling = ChatOpenAI(model=params.tool_calling_model,temperature=0)

## Tavily

In [9]:
from pstuts_rag.agent_tavily import create_tavily_node


adobe_help_node, adobe_help_agent,adobe_search = create_tavily_node(llm=llm_tool_calling,name="AdobeHelp")

In [10]:
adobe_search("What is crop?")

/tmp/ipykernel_7954/3445616894.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  adobe_search("What is crop?")


[{'title': 'Top 4 reasons to crop your photo',
  'url': 'https://helpx.adobe.com/lv/photoshop/how-to/cropping-photo-basics.html',
  'content': 'Cropping is the easiest way to remove unwanted objects or people at the edges of a photograph. Anything outside the crop boundary will disappear from your image',
  'score': 0.585789},
 {'title': 'Crop, resize, and resample images in Photoshop Elements',
  'url': 'https://helpx.adobe.com/photoshop-elements/kb/crop-resize-resample-photoshop-elements.html',
  'content': 'When you crop an image, you trim away material from the edges to show a smaller area, often for artistic reasons.',
  'score': 0.585789},
 {'title': 'How to crop and straighten photos in Photoshop',
  'url': 'https://helpx.adobe.com/photoshop/using/crop-straighten-photos.html',
  'content': "Try it in the app\nYou'll get a sample file to follow along with as you learn how to crop photos.\nOpen Photoshop\nCropping is the process of removing portions of a photo to create focus or s

In [11]:
retval = adobe_help_agent.invoke({"input":"What is crop?","messages":[],"team_members":[],"next":[]}),


In [12]:
from pprint import pp
pp(retval[0]["output"])

('To create a new layer in Adobe Photoshop, you can do any of the following:\n'
 '\n'
 '1. Click the "Create a New Layer" button at the bottom of the Layers panel. '
 'This creates a new transparent layer above the currently selected layer.\n'
 '\n'
 '2. Choose Layer > New > Layer from the top menu. This opens a dialog box '
 'where you can name the layer and set options before creating it.\n'
 '\n'
 '3. Alt-click (Windows) or Option-click (Mac) the "Create a New Layer" button '
 'to open the New Layer dialog box and set layer options.\n'
 '\n'
 '4. Ctrl-click (Windows) or Command-click (Mac) the "Create a New Layer" '
 'button to add a new layer below the currently selected layer.\n'
 '\n'
 'Additionally, you can create a new layer from a selection by making a '
 'selection and then choosing:\n'
 '- Layer > New > Layer Via Copy (copies the selection to a new layer)\n'
 '- Layer > New > Layer Via Cut (cuts the selection and pastes it into a new '
 'layer)\n'
 '\n'
 'A new layer appears

In [13]:
output = adobe_help_node(test_query)

In [14]:
output["messages"][-1].pretty_print()

================================ Human Message =================================
Name: AdobeHelp

Layers in Adobe Photoshop are fundamental elements used for editing and composing images. They are like stacked, transparent sheets of glass, each containing images, text, effects, or objects. Layers allow you to work on different parts of an image independently without affecting other layers. This makes editing nondestructive and flexible.

You can think of layers as separate pieces of content stacked on top of each other. The Layers panel in Photoshop shows these layers arranged in a stack, where you can manage their order, visibility, and properties such as opacity and blending modes. Layers enable you to composite multiple images, add text, apply filters, and make adjustments selectively.

The bottommost layer is usually the Background layer, which is locked by default but can be converted into a regular layer for more editing options.

In summary, layers help you:
- Edit parts of an i

## Data Preparation

First, we will read in the transcripts of the videos and convert them to Documents
with appropriate metadata.

In [15]:
from ast import Dict
import json

from pstuts_rag.loader import load_json_files
filename = ["../data/test.json"]
from typing import List, Dict, Any
data:List[Dict[str,Any]] = await load_json_files(filename)


Let's hit it with a semantic chunker.

In [16]:
from pstuts_rag.datastore import DatastoreManager
from qdrant_client import QdrantClient

client = QdrantClient(path=":memory:")

datastore_manager = DatastoreManager(qdrant_client=client,name="local_test")
if datastore_manager.count_docs() == 0:
    await datastore_manager.populate_database(raw_docs=data)

## RAG Agent

In [17]:
from pstuts_rag.agent_rag import create_rag_node
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
rag_node, rag_search = create_rag_node(retriever=datastore_manager.get_retriever(),
                                         llm=ChatOpenAI(model="gpt-4.1-mini",temperature=0),
                                         name="VideoArchiveSearch" )



<built-in function repr>


In [18]:

retval = rag_search("What is Seinfeld?")


In [19]:
from pprint import pp
pp(retval)

"I don't know. This isn’t covered in the training videos."


In [20]:
rag_output = rag_node(test_query)
rag_output["messages"][-1].pretty_print()

================================ Human Message =================================
Name: VideoArchiveSearch

Layers are the building blocks of any image in Photoshop CC. You can think of layers like separate flat panes of glass stacked on top of each other, with each layer containing separate pieces of content. Some parts of a layer can be transparent, allowing you to see through to the layers below. This setup lets you edit parts of an image independently without affecting the rest of the image. You manage and work with layers in the Layers panel, where you can toggle their visibility on and off using the Eye icon. (See explanation around 0:28 to 1:00 and 1:25 to 2:32) 🎨🖼️
**REFERENCES**
[
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",
    "start": 0.47,
    "stop": 62.14
  },
  {
    "title": "Un

## Graph Creation

In [21]:
from pstuts_rag.agents import create_team_supervisor
from pstuts_rag.prompt_templates import SUPERVISOR_SYSTEM
supervisor_agent = create_team_supervisor(
    llm_tool_calling,
    SUPERVISOR_SYSTEM,
    ["VideoArchiveSearch", "AdobeHelp"],
)



/home/mbudisic/Documents/PsTuts-RAG/pstuts_rag/pstuts_rag/agents.py:79: LangChainDeprecationWarning: The method `BaseChatOpenAI.bind_functions` was deprecated in langchain-openai 0.2.1 and will be removed in 1.0.0. Use :meth:`~langchain_openai.chat_models.base.ChatOpenAI.bind_tools` instead.
  | llm.bind_functions(functions=[function_def], function_call="route")


In [22]:
from langgraph.graph import END, StateGraph

adobe_help_graph = StateGraph(PsTutsTeamState)

adobe_help_graph.add_node("VideoArchiveSearch", rag_node)
adobe_help_graph.add_node("AdobeHelp", adobe_help_node)
adobe_help_graph.add_node("supervisor", supervisor_agent)

edges = [
    ["VideoArchiveSearch","supervisor"],
    ["AdobeHelp","supervisor"],
]

[adobe_help_graph.add_edge(*p) for p in edges]

adobe_help_graph.add_conditional_edges(
    "supervisor",
    lambda x:x["next"],
    {"VideoArchiveSearch":"VideoArchiveSearch",
    "AdobeHelp":"AdobeHelp",    
    "FINISH": END},
)


In [32]:
adobe_help_graph.nodes.keys()

dict_keys(['VideoArchiveSearch', 'AdobeHelp', 'supervisor'])

In [23]:
adobe_help_graph.set_entry_point("supervisor")
compiled_research_graph = adobe_help_graph.compile()

In [24]:
import nest_asyncio
nest_asyncio.apply()

In [25]:
from langchain_core.runnables.graph_ascii import draw_ascii

graph_data = compiled_research_graph.get_graph()
print(graph_data.draw_ascii())

                            +-----------+                             
                            | __start__ |                             
                            +-----------+                             
                                  *                                   
                                  *                                   
                                  *                                   
                            +------------+                            
                            | supervisor |                            
                       *****+------------+.....                       
                   ****            .           ....                   
              *****                .               .....              
           ***                     .                    ...           
+-----------+           +--------------------+           +---------+  
| AdobeHelp |           | VideoArchiveSearch |           | __end__ |  
+-----

In [26]:
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
        "team_members": ["VideoArchiveSearch", "AdobeHelp"],
    }
    return results

research_chain = enter_chain | compiled_research_graph

def demo_research_chain(query:str):
    
    for s in research_chain.stream(
        query, 
        {"recursion_limit": 20}
    ):
        if "__end__" not in s:
            if 'supervisor' not in s.keys():
                for response in s.values():
                    for msg in response['messages']:
                        msg.pretty_print()
            print("---")
            


In [27]:
demo_research_chain("What are layers?")

---
================================ Human Message =================================
Name: VideoArchiveSearch

Layers are the building blocks of any image in Photoshop CC. You can think of layers like separate flat panes of glass stacked on top of each other, where each layer contains separate pieces of content. Some parts of a layer can be transparent, allowing you to see through to the layers below. This setup lets you edit parts of an image independently without affecting the rest of the image. You manage and work with layers in the Layers panel, where you can toggle their visibility on and off using the Eye icon. (See 0:28–1:03 and 1:25–2:32) 🎨🖼️
**REFERENCES**
[
  {
    "title": "Understand layers",
    "source": "https://images-tv.adobe.com/avp/vr/b758b4c4-2a74-41f4-8e67-e2f2eab83c6a/f810fc5b-2b04-4e23-8fa4-5c532e7de6f8/e268fe4d-e5c7-415c-9f5c-d34d024b14d8_20170727011753.1280x720at2400_h264.mp4",
    "start": 0.47,
    "stop": 62.14
  },
  {
    "title": "Understand layers",
    

In [28]:
demo_research_chain("How do we crop a layer?")

---
================================ Human Message =================================
Name: VideoArchiveSearch

I don't know. This isn’t covered in the training videos.
---
---
================================ Human Message =================================
Name: AdobeHelp

To crop a layer in Adobe Photoshop, you can use the Crop tool in a way that targets the active layer or a selection within that layer. Here's how you can do it:

1. Select the layer you want to crop by clicking its thumbnail in the Layers panel.
2. Choose the Crop tool from the toolbar.
3. A bounding box will appear around the active layer or the selection.
4. Drag the handles of the bounding box to set the crop size.
5. You can also rotate the layer by moving the rotation dial below the bounding box if needed.
6. Apply touch gestures or mouse actions to pan, zoom, or reset the crop area.
7. When satisfied, confirm the crop to apply it to the layer.

This method allows you to crop and rotate an active layer or the co